In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")

In [13]:
train = pd.read_parquet('train-00000-of-00001.parquet')
val = pd.read_parquet('validation-00000-of-00001.parquet')

In [14]:
df = pd.concat([train, val], ignore_index=True)

In [15]:
# Get the lists of sentences and their labels.
sentences = df['sentence'].values
labels = df['label'].values

In [16]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [17]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  hide new secretions from the parental units 
Token IDs: [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102]


In [18]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 47...
MAX_LEN = 72

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 72 values...

Padding token: "[PAD]", ID: 0

Done.


In [19]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [20]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [21]:
# Instantiate the random forest model
rf_model = RandomForestClassifier(criterion='gini',max_depth=None, min_samples_leaf=1,n_estimators=100,
                                  max_samples=None,random_state=0)

# Fit the random forest to the data
rf_model.fit(X_train, y_train)

test_preds = rf_model.predict(X_test)
test_acc = np.sum(test_preds==y_test)/len(y_test)
print('Test set accuracy is {:.3f}'.format(test_acc))

Test set accuracy is 0.658


In [24]:
# Set up and run a cross-validated grid search to find the best parameters

from sklearn.model_selection import GridSearchCV

params = {'min_samples_leaf':[1,2,12],
          'max_features':[0.1,0.3,1.]} 

model = RandomForestClassifier()
grid_search = GridSearchCV(model,params,cv=3)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_features': [0.1, 0.3, 1.0],
                         'min_samples_leaf': [1, 2, 12]})

In [25]:
# Display the best parameters
grid_search.best_params_

{'max_features': 1.0, 'min_samples_leaf': 1}

In [26]:
# Instantiate the random forest model
rf_model = RandomForestClassifier(criterion='gini',max_depth=None, 
                                  min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
                                  n_estimators=100,
                                  max_samples=None,
                                  max_features=grid_search.best_params_['max_features'],
                                  random_state=0)

# Fit the random forest to the data
rf_model.fit(X_train, y_train)

test_preds = rf_model.predict(X_test)
test_acc = np.sum(test_preds==y_test)/len(y_test)
print('Test set accuracy is {:.3f}'.format(test_acc))

Test set accuracy is 0.701
